# Prompt Engineering

author: kogni7

date: December 2023 / January 2024

This is a little exercise in Prompt Engineering (PE). We are using Mistral 7B as Large Language Model (LLM).

We will not evaluate the outputs of the LLM. So, the outputs might be incorrect.

References:
* https://www.kaggle.com/models/mistral-ai/mistral/frameworks/pyTorch/variations/7b-instruct-v0.1-hf
* https://www.w3schools.com/gen_ai/chatgpt-4/index.php
* https://learnprompting.org/docs/intro

## Libraries

In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

import re

from transformers.utils import logging
logging.set_verbosity(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

## Model

In [2]:
# taken from https://www.kaggle.com/code/gpreda/multiple-task-chain-with-mistral-model

model_id = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Functions

In [3]:
def generate_output(s):
    messages = [{"role": "user", "content": s + "\n \n"}]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to("cuda")
    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]


def render_output(s):
    print(re.sub(' +', ' ', s.replace("<s>", "").replace("[INST]", "").replace("[/INST]", "").replace("</s>", "")).lstrip())

## ASKING

Let's ask the LLM some questions!

At first, let's ask about the American Actor Steve Carell.

In [4]:
render_output(generate_output("Who is Steve Carell?"))

Who is Steve Carell?
 
 Steve Carell is an American actor and comedian


Now, the LLM shall make a bullet point list of the movies with Steve Carell.

In [5]:
render_output(generate_output("What are the movies of Steve Carell? Make a bullet point list."))

What are the movies of Steve Carell? Make a bullet point list.
 
 Here is a list of some movies starring Steve Carell, in bullet points:

* "The Office" (2005-2013)
* "The Italian Job" (2001)
* "Austin Powers: The Spy Who Shagged Me" (2002)
* "Evan Almighty" (2007)
* "Gregory Coolidge" (2001)
* "The Best Man" (2002)
* "Despicable Me" (2010)
* "The 40-Year-Old Virgin" (2005)
* "Superbad" (2007)
* "Crazy, Stupid, Love" (2011)


Let's check if we can ask questions in another language.

In [6]:
render_output(generate_output("Wer ist Angela Merkel?"))

Wer ist Angela Merkel?
 
 Angela Merkel ist eine deutsche Politikerin und Mitglied der Christlich Demokratischen Union Deutschlands (CDU). Sie war von November 2005 bis November 2021 die Bundeskanzlerin der Bundesrepublik Deutschland und wurde mit einer Amtszeit von 16 Jahren die langste und erstmals überwiegend von einer Frau geführte Bundesregierungschef Deutschlands. Bei den Bundestagswahlen 2021 wurde sie nicht erneut als CDU-Parteivorsitzende gewählt und legte daraufhin am 8. Dezember 2021 ihr Amt als Bundeskanzlerin nieder.


Let's make another bullet point list.

In [7]:
render_output(generate_output("List 10 mediterranian recipes! Make a bullet point list."))

List 10 mediterranian recipes! Make a bullet point list.
 
 1. Mediterranean quinoa salad
2. Veggie-stuffed bell peppers
3. Grilled fish with herbs and lemon
4. Mediterranean chickpea stew
5. Spicy feta dip
6. Couscous with roasted vegetables
7. Greek-style pizza
8. Roasted vegetable and quinoa pizza
9. Mediterranean lentil soup
10. Roasted vegetable and hummus wrap


## WRITING

Let's write an E-Mail to congratulate my grand-father \[this is the action\] to his 85th birthday on Sunday \[this is some context\]. The E-Mail should be a bit funny \[this is some expectation about the output\].

In [8]:
render_output(generate_output("Write an E-Mail to congratulate my grand-father to his 85th birthday on Sunday! The E-Mail should be a bit funny."))

Write an E-Mail to congratulate my grand-father to his 85th birthday on Sunday! The E-Mail should be a bit funny.
 
 Subject: Happy 85th Birthday to Grandpa!

Dear Grandpa,

Congratulations, Grandpa! Your 85th birthday is a major milestone, and you deserve all the recognition! You know what they say, age is just a number, but it must be getting pretty close to zero.

In your entire life, you've gone through so many experiences and met so many people. It's amazing you haven't aged that much, because let's be real, your stories about the past are just as boring as ever.

I hope your birthday is filled with all the things you love: eating too much food, sleeping too much, and telling too many stories. I also hope your family sends you a birthday card that says "Happy 85th Birthday, Gramps!" in all your favorite crayons.

And if all goes well, you'll have two more birthdays after this one. But let's be honest, nobody is making any promises about that.

Happy 85th Birthday, Grandpa! May it 

Are LLMs good poets? Let's check!

In [9]:
render_output(generate_output("Write a short poem about the rain!"))

Write a short poem about the rain!
 
 Soft patters on the window,
The rain is coming down in droplets,
Nature's symphony in full swing,
A soothing lullaby for everything. 

The earth awakens with each drop,
Flowers bloom, trees sway, rivers hop,
The rain washes away all the dirt,
Bringing life and color to everything it touches.

Rain, rain don't go away,
Come again and again to chase the clouds away,
For in your gentle touch, we find peace and calm,
A reminder of nature's healing balm.


## SUMMARIZING

Let's summarize Homer's Odyssey.

In [10]:
render_output(generate_output("Summarize Homer's Odyssey!"))

Summarize Homer's Odyssey!
 
 The Odyssey by Homer is an epic poem that tells the story of Odysseus, a Greek hero, and his journey home to Ithaca after the fall of Troy. On his way back, he faces many challenges, including battles with mythical creatures, visits to the underworld, and temptations by the sirens and the Cyclops. Along the way, he loses all of his crew to the wrath of the gods, except for one loyal slave named Polyphemus.

Finally, after ten years of wandering, Odysseus returns to Ithaca, where he disguises himself as a beggar and with the help of his son Telemachus, he kills the suitors who were trying to steal his wife Penelope's hand in marriage. In the end, he is reunited with his wife and son and they live happily ever after.


Now, let's summarize the historical influence of the Odyssey.

In [11]:
render_output(generate_output("Summarize the historical influence of Homer's Odyssey!"))

Summarize the historical influence of Homer's Odyssey!
 
 The Odyssey, an epic poem by Homer, has had a profound historical influence on Western civilization. It is considered one of the greatest works of ancient literature and has shaped the way that many cultures view heroism, adventure, and the journey home. The story of Odysseus and his travels have been retold and adapted countless times, and the characters and themes have become familiar to people all over the world. The Odyssey has also had a significant impact on the development of language and literature, with many words and phrases in English derived from the poem. Overall, the historical influence of the Odyssey cannot be overstated, and it remains a timeless work that continues to captivate and inspire readers today.


## TRANSLATING

LLMs can even translate text.

In [12]:
render_output(generate_output("Translate the following to German: This is a beautiful house!"))

Translate the following to German: This is a beautiful house!
 
 "Dies ist ein schönes Haus!"


In [13]:
render_output(generate_output("Translate the following to French: This is a beautiful house!"))

Translate the following to French: This is a beautiful house!
 
 Ceci est une maison jolie!


## EXPLAINING

Let's explain Relativity!

In [14]:
render_output(generate_output("Explain Relativity!"))

Explain Relativity!
 
 Relativity is a theory in physics that deals with the way space and time are interrelated. The theory was developed by Albert Einstein in 1905 and 1915. There are two separate theories of relativity, special relativity and general relativity.

Special relativity is based on two postulates: the laws of physics are the same for all non-accelerating observers, and the speed of light is always the same for all observers, regardless of their motion or the motion of the source of the light. These two postulates lead to a number of strange and counter-intuitive consequences, including time dilation, length contraction, and the equivalence of mass and energy (E=mc^2).

General relativity, on the other hand, is a theory of gravity in the context of the relativity of simultaneity. It describes gravity not as a force between masses, but as a curvature of spacetime caused by the presence of mass or energy. According to this theory, the paths of objects in the universe are de

Now, explain Relativity like I'm 5 years old.

In [15]:
render_output(generate_output("Explain Relativity like I'm 5 years old!"))

Explain Relativity like I'm 5 years old!
 
 Sure, I would be glad to explain relativity to you! So, you know when you are sitting in your car and you see a car driving next to you? That car could be going slower than you or faster than you. The speed at which we see the other car is dependent on your own speed and the position of the other car in the world.

Now imagine you are on a train that is moving at a constant speed. And if you are looking out of the train's window, you can see a tree outside. If you want to throw a ball back and forth with the tree, then the tree will appear to you as stationary. That means it will not move back towards you as it receives the ball.

This is the idea behind Einstein's theory of relativity, which says that everything in the universe is relative to something else. This means that space and time can change depending on your speed, the distance you are from an object, and the object's movement. To put it in simple terms, relativity means that things

And now, let's try some role prompting: Acting as a Physics teacher, explain Relativity!

In [16]:
render_output(generate_output("Act as a Physics teacher. Explain Relativity!"))

Act as a Physics teacher. Explain Relativity!
 
 Sure! I'll do my best to explain relativity to you as a physics teacher.

There are two theories of relativity - special relativity and general relativity.

* Special relativity, first proposed by Albert Einstein in 1905, was introduced to account for the speed of light being constant for all observers, regardless of their motion or the motion of the light source. It has four basic principles:

1. the laws of physics are the same for all observers in uniform motion relative to one another (the principle of relativity);
2. the speed of light is always the same in a vacuum, regardless of the motion of its source or the observer;
3. the distance between two points in space is the same for all observers, even if they are in relative motion; and
4. time and space are intertwined into a four-dimensional structure called space-time, and the passage of time can vary depending on the relative motion of a time-moving (non-stationary) object, leadi

## CODING

Let's write a program that just consists of a function $f(x)=x + 2$ in Python, R, C and C++.

In [17]:
render_output(generate_output("Write a Python program that states the function f(x)=x+2"))

Write a Python program that states the function f(x)=x+2
 
 Here's a program in Python that defines the function f(x) = x + 2:

```python
def f(x):
 return x + 2
```

This can be used to calculate the value of the function for any given input `x`. For example, to find the value of `f(3)`, you can input `3` as an argument to the function like this:

```python
result=f(3)
```

The variable `result` would be initialized with the value `5`, since `f(3) = 3 + 2 = 5`. 

Here's an example of how you can use the defined function:

```python
x = 5
y = f(x)

print(y)
```

This would output the value of `y`, which is the result of the function `f()` applied to the input `x`, and is equal to `7` in this case.


In [18]:
render_output(generate_output("Write a R program that states the function f(x)=x+2"))

Write a R program that states the function f(x)=x+2
 
 Here is a simple R program that states the function f(x) = x + 2. 

```R
# Function definition
f <- function(x) {
 return(x + 2)
}

# Test the function
print(f(3)) # Output will be 5
```


In [19]:
render_output(generate_output("Write a C program that states the function f(x)=x+2"))

Write a C program that states the function f(x)=x+2
 
 ```c
#include <stdio.h>

// Function definition
float f(float x){
 return x + 2;
}

int main()
{
 // Example usage of the function
 printf("The value of f(5) is %f", f(5));

 return 0;
}
```
Note: This is a very simple program that states the function `f(x) = x + 2` and provides an example of how to use it. It returns the value of `f(5)` which is `7.000000`.


In [20]:
render_output(generate_output("Write a C++ program that states the function f(x)=x+2"))

Write a C++ program that states the function f(x)=x+2
 
 Here's a C++ program that defines a function, f(x) = x + 2:

```cpp
#include <iostream>
using namespace std;

int main(){
 // Define the function
 double f(double x){
 return x + 2;
 }
 // Call the function and print the result
 double value = f(5);
 cout << "f(5) = " << value << endl; // 7
 return 0;
}
```

This program adds 2 to the input value `x` and returns the result. The function is defined inside the `main` function using the `double` data type, which allows for decimal values. The function is called with the input value `5` and the result is stored in the variable `value`. Finally, the result is printed to the console using the `cout` statement.


## DEBUGGING

Let's ask the LLM what is wrong about:

```
def p():
    return x**2
p(0)
```

In [21]:
render_output(generate_output(
    """
    This is a Python program:
    
    def p():
        return x**2
    p(0)
    
    The program does not work. It throws the following error message:
    
    Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    TypeError: p() takes 0 positional arguments but 1 was given
    
    What is the mistake?
    """))

This is a Python program:
 
 def p():
 return x**2
 p(0)
 
 The program does not work. It throws the following error message:
 
 Traceback (most recent call last):
 File "<stdin>", line 1, in <module>
 TypeError: p() takes 0 positional arguments but 1 was given
 
 What is the mistake?
 
 
 The mistake is that the function `p()` takes zero positional arguments, but you called it with a single argument `x`. To fix the error, you can update the function to accept the argument `x` and then remove it from the function call when you call it. Here's an updated version of the program:

```python
def p(x):
 return x**2

result = p(0)
print(result)
```

This should output `0` without any errors.


## CALCULATING

LLMs can calculate a little bit. Let's try it using some examples.

In [22]:
render_output(generate_output(
    """
    2 + 2 = 4
    3 + 3 = 6
    4 + 4 = 8
    23 + 57 =
    """))

2 + 2 = 4
 3 + 3 = 6
 4 + 4 = 8
 23 + 57 =
 
 
 The answer to the equation 23 + 57 is 80.


## CREATING TABLES

Let's give the LLM a text and ask the LLM to create a table from the text.

In [23]:
render_output(generate_output(
    """
    Apples are red. Oranges are orange. Bananas are yellow. 
    
    Make a table with two columns consisting the above information. The names of the columns are fruit and color.
    """))

Apples are red. Oranges are orange. Bananas are yellow. 
 
 Make a table with two columns consisting the above information. The names of the columns are fruit and color.
 
 
 | Fruit | Color |
|--------|--------|
| Apples | Red |
| Oranges | Orange |
| Bananas | Yellow |
